In [3]:
# Carregar os dados do modelo SA
path_sa = r'C:\Users\Fernando Alves\Documents\WANG PIPELINE\Wang\notebooks\summary_results_effnet\0618-231213_EFF-NET_fold5_results.csv'
df_sa = pd.read_csv(path_sa)  

# Extrair patient_id e tile_name do image_path no SA usando regex
df_sa[['patient_id', 'tile_name']] = df_sa['image_path'].str.extract(r'(TCGA-[^-]+-[^-]+).*\/([^\/]+\.jpeg)')

# Parsear probability_vector e obter a probabilidade da classe verdadeira para SA
df_sa['prob_vector_sa'] = df_sa['probability_vector'].apply(ast.literal_eval)
df_sa['true_prob_sa'] = df_sa.apply(lambda row: row['prob_vector_sa'][row['true_label']], axis=1)

# Agrupar por paciente e calcular a média da probabilidade da classe verdadeira
sa_grouped = df_sa.groupby('patient_id')['true_prob_sa'].mean().reset_index()

In [4]:
sa_grouped

,patient_id,true_prob_sa
0,TCGA-3M-AB46,0.792122
1,TCGA-3M-AB47,0.113637
2,TCGA-B7-A5TI,0.055886
3,TCGA-B7-A5TN,0.607237
4,TCGA-BR-4191,0.660522
...,...,...
78,TCGA-RD-A8N2,0.048525
79,TCGA-VQ-A8PB,0.123806
80,TCGA-VQ-A8PF,0.504888
81,TCGA-VQ-A8PX,0.086565


In [5]:

path_ma = r"C:\Users\Fernando Alves\Documents\WANG PIPELINE\Wang\tables\MOBNet\Ensemble_patient_level_soft_voting\ensemble_per_patient_soft_voting.csv"
# Carregar os dados do modelo MA
df_ma = pd.read_csv(path_ma)

# Parsear mean_probs_per_class_paciente e obter a probabilidade da classe verdadeira para MA
df_ma['prob_vector_ma'] = df_ma['mean_probs_per_class_paciente'].apply(ast.literal_eval)
df_ma['true_prob_ma'] = df_ma.apply(lambda row: row['prob_vector_ma'][row['true_label']], axis=1)

# Agrupar por paciente e calcular a média da probabilidade da classe verdadeira
ma_grouped = df_ma.groupby('patient_id')['true_prob_ma'].mean().reset_index()


In [6]:
ma_grouped

,patient_id,true_prob_ma
0,TCGA-3M-AB47,0.102192
1,TCGA-B7-A5TN,0.622546
2,TCGA-BR-4253,0.317639
3,TCGA-BR-4279,0.605457
4,TCGA-BR-4357,0.543170
...,...,...
77,TCGA-VQ-A8PX,0.314009
78,TCGA-VQ-A91W,0.132300
79,TCGA-VQ-A923,0.346083
80,TCGA-VQ-A928,0.169799


In [7]:

# Unir os dataframes agrupados por patient_id
df_common = pd.merge(sa_grouped, ma_grouped, on='patient_id', how='inner')


In [8]:
df_common

,patient_id,true_prob_sa,true_prob_ma
0,TCGA-3M-AB47,0.113637,0.102192
1,TCGA-B7-A5TN,0.607237,0.622546
2,TCGA-BR-4253,0.179233,0.317639
3,TCGA-BR-4279,0.606522,0.605457
4,TCGA-BR-4361,0.308012,0.321253
5,TCGA-BR-4370,0.299520,0.437798
6,TCGA-BR-7707,0.244372,0.251558
7,TCGA-BR-7715,0.670790,0.705697
8,TCGA-BR-7957,0.479163,0.394065
9,TCGA-BR-8590,0.450435,0.559958


In [9]:
# Verificar se há pacientes comuns
if df_common.empty:
    print("Não há pacientes comuns para comparação.")
else:
    # Calcular estatísticas descritivas para a Tabela 1
    sa_stats = {
        'Número de Pacientes': len(df_common),
        'Média': np.mean(df_common['true_prob_sa']),
        'Desvio Padrão': np.std(df_common['true_prob_sa'], ddof=1),
        'Mediana': np.median(df_common['true_prob_sa']),
        'Mínimo': np.min(df_common['true_prob_sa']),
        'Máximo': np.max(df_common['true_prob_sa'])
    }
    
    ma_stats = {
        'Número de Pacientes': len(df_common),
        'Média': np.mean(df_common['true_prob_ma']),
        'Desvio Padrão': np.std(df_common['true_prob_ma'], ddof=1),
        'Mediana': np.median(df_common['true_prob_ma']),
        'Mínimo': np.min(df_common['true_prob_ma']),
        'Máximo': np.max(df_common['true_prob_ma'])
    }
    
    # Aplicar o teste de Wilcoxon Signed-Rank Test
    stat, p_value = wilcoxon(df_common['true_prob_sa'], df_common['true_prob_ma'])
    
    # Determinar significância estatística
    alpha = 0.05
    significancia = "Sim" if p_value < alpha else "Não"
    
    # Imprimir Tabela 1
    print("Tabela 1. Estatísticas Descritivas das Probabilidades da Classe Verdadeira por Paciente")
    print("Modelo\tNúmero de Pacientes\tMédia\tDesvio Padrão\tMediana\tMínimo\tMáximo")
    print(f"SA ({single_model})\t{sa_stats['Número de Pacientes']}\t{sa_stats['Média']:.4f}\t{sa_stats['Desvio Padrão']:.4f}\t{sa_stats['Mediana']:.4f}\t{sa_stats['Mínimo']:.4f}\t{sa_stats['Máximo']:.4f}")
    print(f"MA (MobileNet Ensemble)\t{ma_stats['Número de Pacientes']}\t{ma_stats['Média']:.4f}\t{ma_stats['Desvio Padrão']:.4f}\t{ma_stats['Mediana']:.4f}\t{ma_stats['Mínimo']:.4f}\t{ma_stats['Máximo']:.4f}")
    
    # Imprimir Tabela 2
    print("\nTabela 2. Resultado do Teste de Wilcoxon Signed-Rank")
    print("Estatística W\tValor-p\tSignificância Estatística (α = 0,05)")
    print(f"{stat:.4f}\t{p_value:.5f}\t{significancia}")
    
    # Interpretação adicional
    print(f"\nInterpretação:")
    if p_value < alpha:
        print("Há uma diferença estatisticamente significativa entre os modelos (rejeitamos H0).")
        if sa_stats['Média'] > ma_stats['Média']:
            print(f"O modelo SA ({single_model}) tem desempenho superior.")
        else:
            print("O modelo MA (MobileNet ensemble) tem desempenho superior.")
    else:
        print("Não há diferença estatisticamente significativa entre os modelos (não rejeitamos H0).")

Tabela 1. Estatísticas Descritivas das Probabilidades da Classe Verdadeira por Paciente
Modelo	Número de Pacientes	Média	Desvio Padrão	Mediana	Mínimo	Máximo
SA (EfficientNet)	32	0.3432	0.2350	0.2975	0.0295	0.8470
MA (MobileNet Ensemble)	32	0.4375	0.1944	0.4588	0.1022	0.7466

Tabela 2. Resultado do Teste de Wilcoxon Signed-Rank
Estatística W	Valor-p	Significância Estatística (α = 0,05)
105.0000	0.00223	Sim

Interpretação:
Há uma diferença estatisticamente significativa entre os modelos (rejeitamos H0).
O modelo MA (MobileNet ensemble) tem desempenho superior.


## **Por Paciente**

In [73]:
import pandas as pd
from scipy.stats import wilcoxon
import ast
import numpy as np
import os
from collections import Counter
from sklearn.metrics import precision_score, recall_score, f1_score, classification_report
from sklearn.utils import resample

### **Funções Uteis**

In [74]:
# Função para processar os dados do SA (EffNet)
def process_sa_data(folds_dir):
    """Processa todos os folds do modelo SA e retorna um DataFrame consolidado"""
    all_sa_data = []
    
    # Listar todos os arquivos de fold
    fold_files = [f for f in os.listdir(folds_dir) if f.endswith('_results.csv')]
    
    for fold_file in fold_files:
        fold_path = os.path.join(folds_dir, fold_file)
        df_fold = pd.read_csv(fold_path)
        
        # Extrair patient_id e tile_name usando regex
        df_fold[['patient_id', 'tile_name']] = df_fold['image_path'].str.extract(
            r'(TCGA-[^-]+-[^-]+).*\/([^\/]+\.jpeg)'
        )
        
        # Parsear probability_vector e obter a probabilidade da classe verdadeira
        df_fold['prob_vector'] = df_fold['probability_vector'].apply(ast.literal_eval)
        df_fold['true_prob'] = df_fold.apply(
            lambda row: row['prob_vector'][row['true_label']], axis=1
        )
        
        # Adicionar identificador do fold
        df_fold['fold'] = fold_file.split('_fold')[1].split('_')[0]
        
        all_sa_data.append(df_fold)
    
    # Consolidar todos os dados
    df_sa = pd.concat(all_sa_data, ignore_index=True)
    return df_sa

# Função para construir a tabela ensemble a partir das três arquiteturas
def build_ensemble_table(models_dir):
    """Constrói a tabela ensemble a partir dos arquivos das três arquiteturas"""
    # Mapeamento de arquiteturas e seus prefixos
    architectures = {
        'gg': {'file': 'merged_table_gg.csv', 'prefix': 'g'},
        'mob': {'file': 'merged_table_mob.csv', 'prefix': 'm'}, 
        'sh': {'file': 'merged_table_sh.csv', 'prefix': 's'}
    }
    
    # Carregar e processar cada arquitetura
    all_models_data = []
    
    for arch_name, arch_info in architectures.items():
        file_path = os.path.join(models_dir, arch_info['file'])
        prefix = arch_info['prefix']
        
        if os.path.exists(file_path):
            print(f"Processando {arch_name}...")
            df_arch = pd.read_csv(file_path)
            
            # Extrair patient_id e tile_name
            df_arch[['patient_id', 'tile_name']] = df_arch['image_path'].str.extract(
                r'(TCGA-[^-]+-[^-]+).*\/([^\/]+\.jpeg)'
            )
            
            # Para cada fold (0 a 9) na arquitetura, extrair probabilidades
            for i in range(10):
                prob_col = f'probability_vector_{prefix}{i}'
                if prob_col in df_arch.columns:
                    df_arch[prob_col] = df_arch[prob_col].apply(ast.literal_eval)
            
            all_models_data.append(df_arch)
    
    # Combinar todos os dados
    if not all_models_data:
        raise ValueError("Nenhum dado de arquitetura foi encontrado")
    
    # Usar o primeiro DataFrame como base
    df_ensemble = all_models_data[0][['patient_id', 'tile_name', 'true_label']].copy()
    
    # Coletar todas as probabilidades de todos os modelos
    all_probs = []
    for df_arch in all_models_data:
        for i in range(10):
            for prefix in ['g', 'm', 's']:
                prob_col = f'probability_vector_{prefix}{i}'
                if prob_col in df_arch.columns:
                    # Adicionar as probabilidades desta coluna
                    probs_series = df_arch[prob_col].apply(
                        lambda x: x if isinstance(x, dict) else {}
                    )
                    all_probs.append(probs_series)
    
    # Calcular probabilidades médias por classe para cada tile
    print("Calculando probabilidades médias...")
    mean_probs_list = []
    true_probs_list = []
    predicted_labels_list = []
    predicted_probs_list = []
    
    for idx in range(len(df_ensemble)):
        # Coletar todas as probabilidades para este tile
        tile_probs = []
        for probs_series in all_probs:
            if idx < len(probs_series) and probs_series.iloc[idx]:
                tile_probs.append(probs_series.iloc[idx])
        
        if tile_probs:
            # Calcular médias por classe
            mean_probs = {
                'cin': np.mean([p.get('cin', 0) for p in tile_probs]),
                'ebv': np.mean([p.get('ebv', 0) for p in tile_probs]),
                'msi': np.mean([p.get('msi', 0) for p in tile_probs]),
                'gs': np.mean([p.get('gs', 0) for p in tile_probs])
            }
            
            # Determinar label predito
            predicted_label = max(mean_probs, key=mean_probs.get)
            predicted_prob = mean_probs[predicted_label]
            
            # Obter a true_label
            true_label = df_ensemble.iloc[idx]['true_label']
            true_prob = mean_probs.get(true_label, 0)
            
            mean_probs_list.append(mean_probs)
            true_probs_list.append(true_prob)
            predicted_labels_list.append(predicted_label)
            predicted_probs_list.append(predicted_prob)
        else:
            # Caso não haja probabilidades para este tile
            mean_probs_list.append({})
            true_probs_list.append(0)
            predicted_labels_list.append('')
            predicted_probs_list.append(0)
    
    # Adicionar colunas ao DataFrame ensemble
    df_ensemble['mean_probs_per_class_tile'] = mean_probs_list
    df_ensemble['true_prob'] = true_probs_list
    df_ensemble['predicted_label'] = predicted_labels_list
    df_ensemble['predicted_probability'] = predicted_probs_list
    
    return df_ensemble

# Função para calcular métricas por classe
def calculate_metrics_per_class(df):
    """Calcula métricas de desempenho por classe"""
    # Calcular métricas por classe
    class_report = classification_report(
        df['true_label'], 
        df['predicted_label'], 
        output_dict=True,
        zero_division=0
    )
    
    # Extrair métricas para cada classe
    metrics_per_class = {}
    classes = ['cin', 'ebv', 'msi', 'gs']
    
    for cls in classes:
        if cls in class_report:
            metrics_per_class[cls] = {
                'precision': class_report[cls]['precision'],
                'recall': class_report[cls]['recall'],
                'f1': class_report[cls]['f1-score'],
                'support': class_report[cls]['support']
            }
    
    # Calcular métricas weighted average
    metrics_per_class['weighted'] = {
        'precision': class_report['weighted avg']['precision'],
        'recall': class_report['weighted avg']['recall'],
        'f1': class_report['weighted avg']['f1-score'],
        'support': class_report['weighted avg']['support']
    }
    
    # Calcular acurácia geral
    accuracy = class_report['accuracy']
    
    return {
        'accuracy': accuracy,
        'metrics_per_class': metrics_per_class
    }

# Função para calcular métricas com bootstrapping
def calculate_metrics_with_bootstrapping(df, n_bootstraps=100):
    """Calcula métricas usando bootstrapping para estimar variabilidade"""
    bootstrap_metrics = {
        'accuracy': [],
        'precision_cin': [], 'precision_ebv': [], 'precision_msi': [], 'precision_gs': [], 'precision_weighted': [],
        'recall_cin': [], 'recall_ebv': [], 'recall_msi': [], 'recall_gs': [], 'recall_weighted': [],
        'f1_cin': [], 'f1_ebv': [], 'f1_msi': [], 'f1_gs': [], 'f1_weighted': []
    }
    
    for _ in range(n_bootstraps):
        # Amostra bootstrap com reposição
        bootstrap_sample = resample(df, replace=True, random_state=_)
        
        # Calcular métricas para a amostra bootstrap
        metrics = calculate_metrics_per_class(bootstrap_sample)
        
        # Armazenar métricas
        bootstrap_metrics['accuracy'].append(metrics['accuracy'])
        
        for cls in ['cin', 'ebv', 'msi', 'gs', 'weighted']:
            if cls in metrics['metrics_per_class']:
                bootstrap_metrics[f'precision_{cls}'].append(metrics['metrics_per_class'][cls]['precision'])
                bootstrap_metrics[f'recall_{cls}'].append(metrics['metrics_per_class'][cls]['recall'])
                bootstrap_metrics[f'f1_{cls}'].append(metrics['metrics_per_class'][cls]['f1'])
    
    # Calcular médias e desvios padrão
    result = {}
    for metric, values in bootstrap_metrics.items():
        if values:  # Verificar se a lista não está vazia
            result[f'{metric}_mean'] = np.mean(values)
            result[f'{metric}_std'] = np.std(values, ddof=1)
    
    return result

# Função para calcular métricas por paciente
def calculate_metrics_per_patient(df, model_name):
    """Calcula métricas de desempenho agrupadas por paciente"""
    grouped = df.groupby('patient_id')
    
    results = []
    for patient, group in grouped:
        # Calcular acurácia
        accuracy = np.mean(group['true_label'] == group['predicted_label'])
        
        # Calcular precision, recall e f1 (usando weighted average)
        from sklearn.metrics import precision_score, recall_score, f1_score
        
        # Para multi-classe, precisamos garantir que todas as classes estejam representadas
        unique_classes = list(set(group['true_label'].unique()) | set(group['predicted_label'].unique()))
        precision = precision_score(group['true_label'], group['predicted_label'], 
                                  average='weighted', zero_division=0, labels=unique_classes)
        recall = recall_score(group['true_label'], group['predicted_label'], 
                             average='weighted', zero_division=0, labels=unique_classes)
        f1 = f1_score(group['true_label'], group['predicted_label'], 
                     average='weighted', zero_division=0, labels=unique_classes)
        
        # Calcular a média da probabilidade da classe verdadeira
        mean_true_prob = group['true_prob'].mean()
        
        results.append({
            'patient_id': patient,
            f'accuracy_{model_name}': accuracy,
            f'precision_{model_name}': precision,
            f'recall_{model_name}': recall,
            f'f1_{model_name}': f1,
            f'true_prob_{model_name}': mean_true_prob
        })
    
    return pd.DataFrame(results)


In [75]:
# Configurações
SA_FOLDS_DIR = r'C:\Users\Fernando Alves\Documents\WANG PIPELINE\Wang\notebooks\summary_results_effnet'
MA_MODELS_DIR = r'C:\Users\Fernando Alves\Documents\WANG PIPELINE\Wang\MA_models'
N_BOOTSTRAPS = 100  # Número de amostras bootstrap para calcular o desvio padrão


In [76]:

# Processar dados
print("Processando dados do SA (EffNet)...")
df_sa = process_sa_data(SA_FOLDS_DIR)

print("Construindo tabela ensemble do MA...")
df_ma = build_ensemble_table(MA_MODELS_DIR)

# Calcular métricas com bootstrapping para o SA
print("Calculando métricas com bootstrapping para o SA...")
sa_bootstrap_metrics = calculate_metrics_with_bootstrapping(df_sa, N_BOOTSTRAPS)

# Calcular métricas com bootstrapping para o MA
print("Calculando métricas com bootstrapping para o MA...")
ma_bootstrap_metrics = calculate_metrics_with_bootstrapping(df_ma, N_BOOTSTRAPS)

# Calcular métricas por paciente para o teste de Wilcoxon
print("Calculando métricas por paciente para o teste de Wilcoxon...")
sa_patient_metrics = calculate_metrics_per_patient(df_sa, 'sa')
ma_patient_metrics = calculate_metrics_per_patient(df_ma, 'ma')

# Unir os dataframes por patient_id
df_common = pd.merge(sa_patient_metrics, ma_patient_metrics, on='patient_id', how='inner')

# Verificar se há pacientes comuns
if df_common.empty:
    print("Não há pacientes comuns para comparação.")
else:
    # Lista de métricas para comparar
    metrics = ['accuracy', 'precision', 'recall', 'f1', 'true_prob']
    
    # Dicionário para armazenar resultados do Wilcoxon
    wilcoxon_results = {}
    
    # Aplicar o teste de Wilcoxon para cada métrica
    for metric in metrics:
        stat, p_value = wilcoxon(df_common[f'{metric}_sa'], df_common[f'{metric}_ma'])
        wilcoxon_results[metric] = {
            'statistic': stat,
            'p_value': p_value,
            'significant': p_value < 0.05
        }
    
    # Calcular estatísticas descritivas para cada métrica
    descriptive_stats = {}
    for metric in metrics:
        descriptive_stats[metric] = {
            'sa_mean': np.mean(df_common[f'{metric}_sa']),
            'sa_std': np.std(df_common[f'{metric}_sa'], ddof=1),
            'ma_mean': np.mean(df_common[f'{metric}_ma']),
            'ma_std': np.std(df_common[f'{metric}_ma'], ddof=1),
        }

# Gerar tabelas para cada métrica (Precision, Recall, F1)
metrics_to_analyze = ['precision', 'recall', 'f1']
classes = ['cin', 'ebv', 'msi', 'gs', 'weighted']

for metric in metrics_to_analyze:
    print(f"\n=== Tabela para {metric.capitalize()} ===")
    print("Class\tSA (EffNet)\tMA (Ensemble)")
    
    for cls in classes:
        sa_mean = sa_bootstrap_metrics.get(f'{metric}_{cls}_mean', np.nan)
        sa_std = sa_bootstrap_metrics.get(f'{metric}_{cls}_std', np.nan)
        ma_mean = ma_bootstrap_metrics.get(f'{metric}_{cls}_mean', np.nan)
        ma_std = ma_bootstrap_metrics.get(f'{metric}_{cls}_std', np.nan)
        
        if not np.isnan(sa_mean) and not np.isnan(ma_mean):
            print(f"{cls}\t{sa_mean:.4f} ± {sa_std:.4f}\t{ma_mean:.4f} ± {ma_std:.4f}")
    
    # Criar DataFrame para exportação
    export_data = []
    for cls in classes:
        sa_mean = sa_bootstrap_metrics.get(f'{metric}_{cls}_mean', np.nan)
        sa_std = sa_bootstrap_metrics.get(f'{metric}_{cls}_std', np.nan)
        ma_mean = ma_bootstrap_metrics.get(f'{metric}_{cls}_mean', np.nan)
        ma_std = ma_bootstrap_metrics.get(f'{metric}_{cls}_std', np.nan)
        
        if not np.isnan(sa_mean) and not np.isnan(ma_mean):
            export_data.append({
                'Class': cls,
                'SA_Mean': sa_mean,
                'SA_Std': sa_std,
                'SA_Formatted': f"{sa_mean:.4f} ± {sa_std:.4f}",
                'MA_Mean': ma_mean,
                'MA_Std': ma_std,
                'MA_Formatted': f"{ma_mean:.4f} ± {ma_std:.4f}"
            })
    
    export_df = pd.DataFrame(export_data)
    
    # Salvar tabela
    output_path = os.path.join(MA_MODELS_DIR, f'{metric}_per_class_comparison_with_std.csv')
    export_df.to_csv(output_path, index=False)
    print(f"Tabela {metric} salva em: {output_path}")

# Calcular e exibir acurácia
print("\n=== Acurácia ===")
sa_accuracy_mean = sa_bootstrap_metrics.get('accuracy_mean', np.nan)
sa_accuracy_std = sa_bootstrap_metrics.get('accuracy_std', np.nan)
ma_accuracy_mean = ma_bootstrap_metrics.get('accuracy_mean', np.nan)
ma_accuracy_std = ma_bootstrap_metrics.get('accuracy_std', np.nan)

if not np.isnan(sa_accuracy_mean) and not np.isnan(ma_accuracy_mean):
    print(f"SA (EffNet) Acurácia: {sa_accuracy_mean:.4f} ± {sa_accuracy_std:.4f}")
    print(f"MA (Ensemble) Acurácia: {ma_accuracy_mean:.4f} ± {ma_accuracy_std:.4f}")

    # Salvar tabela de acurácia
    accuracy_df = pd.DataFrame({
        'Model': ['SA (EffNet)', 'MA (Ensemble)'],
        'Accuracy_Mean': [sa_accuracy_mean, ma_accuracy_mean],
        'Accuracy_Std': [sa_accuracy_std, ma_accuracy_std],
        'Formatted': [
            f"{sa_accuracy_mean:.4f} ± {sa_accuracy_std:.4f}",
            f"{ma_accuracy_mean:.4f} ± {ma_accuracy_std:.4f}"
        ]
    })
    accuracy_output_path = os.path.join(MA_MODELS_DIR, 'accuracy_comparison_with_std.csv')
    accuracy_df.to_csv(accuracy_output_path, index=False)
    print(f"Tabela de acurácia salva em: {accuracy_output_path}")

# Resultados do teste de Wilcoxon
if not df_common.empty:
    print("\n=== Resultados do Teste de Wilcoxon Signed-Rank ===")
    print("Métrica\tEstatística W\tValor-p\tSignificância Estatística (α = 0,05)")
    for metric in metrics:
        sig = "Sim" if wilcoxon_results[metric]['significant'] else "Não"
        print(f"{metric}\t{wilcoxon_results[metric]['statistic']:.4f}\t{wilcoxon_results[metric]['p_value']:.5f}\t{sig}")
    
    # Interpretação adicional
    print(f"\nInterpretação:")
    for metric in metrics:
        if wilcoxon_results[metric]['p_value'] < 0.05:
            if descriptive_stats[metric]['sa_mean'] > descriptive_stats[metric]['ma_mean']:
                print(f"O modelo SA (EffNet) tem desempenho significativamente melhor em {metric}.")
            else:
                print(f"O modelo MA (Multi-Architecture) tem desempenho significativamente melhor em {metric}.")
        else:
            print(f"Não há diferença estatisticamente significativa entre os modelos em {metric}.")

# Salvar a tabela ensemble para uso futuro
ensemble_output_path = os.path.join(MA_MODELS_DIR, 'ensemble_per_tile_soft_voting.csv')
df_ma.to_csv(ensemble_output_path, index=False)
print(f"\nTabela ensemble salva em: {ensemble_output_path}")

# Gerar relatório completo
print("\n=== Relatório Completo ===")
print("Métrica\tClasse\tSA (EffNet)\tMA (Ensemble)")

for metric in metrics_to_analyze:
    for cls in classes:
        sa_mean = sa_bootstrap_metrics.get(f'{metric}_{cls}_mean', np.nan)
        sa_std = sa_bootstrap_metrics.get(f'{metric}_{cls}_std', np.nan)
        ma_mean = ma_bootstrap_metrics.get(f'{metric}_{cls}_mean', np.nan)
        ma_std = ma_bootstrap_metrics.get(f'{metric}_{cls}_std', np.nan)
        
        if not np.isnan(sa_mean) and not np.isnan(ma_mean):
            print(f"{metric}\t{cls}\t{sa_mean:.4f} ± {sa_std:.4f}\t{ma_mean:.4f} ± {ma_std:.4f}")

if not np.isnan(sa_accuracy_mean) and not np.isnan(ma_accuracy_mean):
    print(f"accuracy\toverall\t{sa_accuracy_mean:.4f} ± {sa_accuracy_std:.4f}\t{ma_accuracy_mean:.4f} ± {ma_accuracy_std:.4f}")

Processando dados do SA (EffNet)...
Construindo tabela ensemble do MA...
Processando gg...
Processando mob...
Processando sh...
Calculando probabilidades médias...
Calculando métricas com bootstrapping para o SA...
Calculando métricas com bootstrapping para o MA...
Calculando métricas por paciente para o teste de Wilcoxon...


c:\Users\Fernando Alves\anaconda3\envs\redes\lib\site-packages\scipy\stats\_wilcoxon.py:172: RuntimeWarning: invalid value encountered in scalar divide
  z = (r_plus - mn) / se



=== Tabela para Precision ===
Class	SA (EffNet)	MA (Ensemble)
cin	0.5388 ± 0.0006	0.6248 ± 0.0014
ebv	0.1030 ± 0.0007	0.6242 ± 0.0037
msi	0.3844 ± 0.0008	0.6450 ± 0.0038
gs	0.1650 ± 0.0008	0.3362 ± 0.0027
weighted	0.3840 ± 0.0005	0.5853 ± 0.0013
Tabela precision salva em: C:\Users\Fernando Alves\Documents\WANG PIPELINE\Wang\MA_models\precision_per_class_comparison_with_std.csv

=== Tabela para Recall ===
Class	SA (EffNet)	MA (Ensemble)
cin	0.5968 ± 0.0005	0.8048 ± 0.0014
ebv	0.0957 ± 0.0007	0.3906 ± 0.0030
msi	0.3739 ± 0.0007	0.2819 ± 0.0023
gs	0.1305 ± 0.0006	0.3970 ± 0.0031
weighted	0.4014 ± 0.0004	0.5751 ± 0.0011
Tabela recall salva em: C:\Users\Fernando Alves\Documents\WANG PIPELINE\Wang\MA_models\recall_per_class_comparison_with_std.csv

=== Tabela para F1 ===
Class	SA (EffNet)	MA (Ensemble)
cin	0.5663 ± 0.0005	0.7035 ± 0.0012
ebv	0.0992 ± 0.0007	0.4805 ± 0.0029
msi	0.3791 ± 0.0006	0.3923 ± 0.0026
gs	0.1458 ± 0.0007	0.3641 ± 0.0026
weighted	0.3917 ± 0.0004	0.5554 ± 0.0012
Tabela 

In [77]:

# Configurações
SA_FOLDS_DIR = r'C:\Users\Fernando Alves\Documents\WANG PIPELINE\Wang\notebooks\summary_results_effnet'
MA_MODELS_DIR = r'C:\Users\Fernando Alves\Documents\WANG PIPELINE\Wang\MA_models'


In [78]:
# Função para processar os dados do SA (EffNet)
def process_sa_data(folds_dir):
    """Processa todos os folds do modelo SA e retorna um DataFrame consolidado"""
    all_sa_data = []
    
    # Listar todos os arquivos de fold
    fold_files = [f for f in os.listdir(folds_dir) if f.endswith('_results.csv')]
    
    for fold_file in fold_files:
        fold_path = os.path.join(folds_dir, fold_file)
        df_fold = pd.read_csv(fold_path)
        
        # Extrair patient_id e tile_name usando regex
        df_fold[['patient_id', 'tile_name']] = df_fold['image_path'].str.extract(
            r'(TCGA-[^-]+-[^-]+).*\/([^\/]+\.jpeg)'
        )
        
        # Parsear probability_vector e obter a probabilidade da classe verdadeira
        df_fold['prob_vector'] = df_fold['probability_vector'].apply(ast.literal_eval)
        df_fold['true_prob'] = df_fold.apply(
            lambda row: row['prob_vector'][row['true_label']], axis=1
        )
        
        # Adicionar identificador do fold
        df_fold['fold'] = fold_file.split('_fold')[1].split('_')[0]
        
        all_sa_data.append(df_fold)
    
    # Consolidar todos os dados
    df_sa = pd.concat(all_sa_data, ignore_index=True)
    return df_sa

# Função para construir a tabela ensemble a partir das três arquiteturas
def build_ensemble_table(models_dir):
    """Constrói a tabela ensemble a partir dos arquivos das três arquiteturas"""
    # Mapeamento de arquiteturas e seus prefixos
    architectures = {
        'gg': {'file': 'merged_table_gg.csv', 'prefix': 'g'},
        'mob': {'file': 'merged_table_mob.csv', 'prefix': 'm'}, 
        'sh': {'file': 'merged_table_sh.csv', 'prefix': 's'}
    }
    
    # Carregar e processar cada arquitetura
    all_models_data = []
    
    for arch_name, arch_info in architectures.items():
        file_path = os.path.join(models_dir, arch_info['file'])
        prefix = arch_info['prefix']
        
        if os.path.exists(file_path):
            print(f"Processando {arch_name}...")
            df_arch = pd.read_csv(file_path)
            
            # Extrair patient_id e tile_name
            df_arch[['patient_id', 'tile_name']] = df_arch['image_path'].str.extract(
                r'(TCGA-[^-]+-[^-]+).*\/([^\/]+\.jpeg)'
            )
            
            # Para cada fold (0 a 9) na arquitetura, extrair probabilidades
            for i in range(10):
                prob_col = f'probability_vector_{prefix}{i}'
                if prob_col in df_arch.columns:
                    df_arch[prob_col] = df_arch[prob_col].apply(ast.literal_eval)
            
            all_models_data.append(df_arch)
    
    # Combinar todos os dados
    if not all_models_data:
        raise ValueError("Nenhum dado de arquitetura foi encontrado")
    
    # Usar o primeiro DataFrame como base
    df_ensemble = all_models_data[0][['patient_id', 'tile_name', 'true_label']].copy()
    
    # Coletar todas as probabilidades de todos os modelos
    all_probs = []
    for df_arch in all_models_data:
        for i in range(10):
            for prefix in ['g', 'm', 's']:
                prob_col = f'probability_vector_{prefix}{i}'
                if prob_col in df_arch.columns:
                    # Adicionar as probabilidades desta coluna
                    probs_series = df_arch[prob_col].apply(
                        lambda x: x if isinstance(x, dict) else {}
                    )
                    all_probs.append(probs_series)
    
    # Calcular probabilidades médias por classe para cada tile
    print("Calculando probabilidades médias...")
    mean_probs_list = []
    true_probs_list = []
    predicted_labels_list = []
    predicted_probs_list = []
    
    for idx in range(len(df_ensemble)):
        # Coletar todas as probabilidades para este tile
        tile_probs = []
        for probs_series in all_probs:
            if idx < len(probs_series) and probs_series.iloc[idx]:
                tile_probs.append(probs_series.iloc[idx])
        
        if tile_probs:
            # Calcular médias por classe
            mean_probs = {
                'cin': np.mean([p.get('cin', 0) for p in tile_probs]),
                'ebv': np.mean([p.get('ebv', 0) for p in tile_probs]),
                'msi': np.mean([p.get('msi', 0) for p in tile_probs]),
                'gs': np.mean([p.get('gs', 0) for p in tile_probs])
            }
            
            # Determinar label predito
            predicted_label = max(mean_probs, key=mean_probs.get)
            predicted_prob = mean_probs[predicted_label]
            
            # Obter a true_label
            true_label = df_ensemble.iloc[idx]['true_label']
            true_prob = mean_probs.get(true_label, 0)
            
            mean_probs_list.append(mean_probs)
            true_probs_list.append(true_prob)
            predicted_labels_list.append(predicted_label)
            predicted_probs_list.append(predicted_prob)
        else:
            # Caso não haja probabilidades para este tile
            mean_probs_list.append({})
            true_probs_list.append(0)
            predicted_labels_list.append('')
            predicted_probs_list.append(0)
    
    # Adicionar colunas ao DataFrame ensemble
    df_ensemble['mean_probs_per_class_tile'] = mean_probs_list
    df_ensemble['true_prob'] = true_probs_list
    df_ensemble['predicted_label'] = predicted_labels_list
    df_ensemble['predicted_probability'] = predicted_probs_list
    
    return df_ensemble

# Função para calcular métricas a nível de tile
def calculate_tile_level_metrics(df, model_name):
    """Calcula métricas de desempenho a nível de tile"""
    # Para cada tile, determinar se a previsão está correta
    df['correct'] = df['true_label'] == df['predicted_label']
    
    # Calcular métricas
    accuracy = np.mean(df['correct'])
    
    # Calcular precision, recall e f1
    precision = precision_score(df['true_label'], df['predicted_label'], 
                              average='weighted', zero_division=0)
    recall = recall_score(df['true_label'], df['predicted_label'], 
                         average='weighted', zero_division=0)
    f1 = f1_score(df['true_label'], df['predicted_label'], 
                 average='weighted', zero_division=0)
    
    # Calcular a média da probabilidade da classe verdadeira
    mean_true_prob = df['true_prob'].mean()
    
    return {
        'model_name': model_name,
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1': f1,
        'true_prob': mean_true_prob,
        'tile_count': len(df)
    }



In [82]:
# Função auxiliar para formatar valores
def format_value(value, format_str="{:.4f}"):
    if isinstance(value, (int, float, np.number)):
        return format_str.format(value)
    else:
        return str(value)

# Função específica para formatar valor-p com mais casas decimais
def format_p_value(value):
    if isinstance(value, (int, float, np.number)):
        return "{:.6f}".format(value)
    else:
        return str(value)

In [83]:
# Na tabela Precision:
print(f"{cls}\t\t{format_value(sa_mean)} ± {format_value(sa_std)}\t{format_value(ma_mean)} ± {format_value(ma_std)}\t{format_value(stat)}\t{format_p_value(p_value)}\t{sig}")

# Na tabela Recall:
print(f"{cls}\t\t{format_value(sa_mean)} ± {format_value(sa_std)}\t{format_value(ma_mean)} ± {format_value(ma_std)}\t{format_value(stat)}\t{format_p_value(p_value)}\t{sig}")

# Na tabela F1-Score:
print(f"{cls}\t\t{format_value(sa_mean)} ± {format_value(sa_std)}\t{format_value(ma_mean)} ± {format_value(ma_std)}\t{format_value(stat)}\t{format_p_value(p_value)}\t{sig}")

# Na tabela Acurácia e True_prob:
print(f"Acurácia\tOverall\t\t{format_value(sa_acc_mean)} ± {format_value(sa_acc_std)}\t{format_value(ma_acc_mean)} ± {format_value(ma_acc_std)}\t{format_value(stat_acc)}\t{format_p_value(p_value_acc)}\t{sig_acc}")

print(f"True_prob\tOverall\t\t{format_value(sa_true_prob_mean)} ± {format_value(sa_true_prob_std)}\t{format_value(ma_true_prob_mean)} ± {format_value(ma_true_prob_std)}\t{format_value(stat_true_prob)}\t{format_p_value(p_value_true_prob)}\t{sig_true_prob}")

# E para true_prob por classe:
print(f"True_prob\t{cls}\t\t{format_value(sa_mean)} ± {format_value(sa_std)}\t{format_value(ma_mean)} ± {format_value(ma_std)}\t{format_value(stat)}\t{format_p_value(p_value)}\t{sig}")

gs		0.1071 ± 0.1764	0.2680 ± 0.1781	64673041.0000	0.000000	Sim
gs		0.1071 ± 0.1764	0.2680 ± 0.1781	64673041.0000	0.000000	Sim
gs		0.1071 ± 0.1764	0.2680 ± 0.1781	64673041.0000	0.000000	Sim
Acurácia	Overall		0.4014 ± 0.0004	0.5751 ± 0.0011	3186877723.0000	0.000000	Sim
True_prob	Overall		0.4784 ± 0.3421	0.4670 ± 0.2262	41720264207.5000	0.000000	Sim
True_prob	gs		0.1071 ± 0.1764	0.2680 ± 0.1781	64673041.0000	0.000000	Sim


In [84]:
import pandas as pd
from scipy.stats import wilcoxon
import ast
import numpy as np
import os
from collections import Counter
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score
from sklearn.utils import resample

# Configurações
SA_FOLDS_DIR = r'C:\Users\Fernando Alves\Documents\WANG PIPELINE\Wang\notebooks\summary_results_effnet'
MA_MODELS_DIR = r'C:\Users\Fernando Alves\Documents\WANG PIPELINE\Wang\MA_models'
N_BOOTSTRAPS = 100  # Número de amostras bootstrap

# Funções para processar dados (mantidas como no código anterior)
def process_sa_data(folds_dir):
    """Processa todos os folds do modelo SA e retorna um DataFrame consolidado"""
    all_sa_data = []
    
    fold_files = [f for f in os.listdir(folds_dir) if f.endswith('_results.csv')]
    
    for fold_file in fold_files:
        fold_path = os.path.join(folds_dir, fold_file)
        df_fold = pd.read_csv(fold_path)
        
        df_fold[['patient_id', 'tile_name']] = df_fold['image_path'].str.extract(
            r'(TCGA-[^-]+-[^-]+).*\/([^\/]+\.jpeg)'
        )
        
        df_fold['prob_vector'] = df_fold['probability_vector'].apply(ast.literal_eval)
        df_fold['true_prob'] = df_fold.apply(
            lambda row: row['prob_vector'][row['true_label']], axis=1
        )
        
        df_fold['fold'] = fold_file.split('_fold')[1].split('_')[0]
        all_sa_data.append(df_fold)
    
    df_sa = pd.concat(all_sa_data, ignore_index=True)
    return df_sa

def build_ensemble_table(models_dir):
    """Constrói a tabela ensemble a partir dos arquivos das três arquiteturas"""
    architectures = {
        'gg': {'file': 'merged_table_gg.csv', 'prefix': 'g'},
        'mob': {'file': 'merged_table_mob.csv', 'prefix': 'm'}, 
        'sh': {'file': 'merged_table_sh.csv', 'prefix': 's'}
    }
    
    all_models_data = []
    
    for arch_name, arch_info in architectures.items():
        file_path = os.path.join(models_dir, arch_info['file'])
        prefix = arch_info['prefix']
        
        if os.path.exists(file_path):
            df_arch = pd.read_csv(file_path)
            
            df_arch[['patient_id', 'tile_name']] = df_arch['image_path'].str.extract(
                r'(TCGA-[^-]+-[^-]+).*\/([^\/]+\.jpeg)'
            )
            
            for i in range(10):
                prob_col = f'probability_vector_{prefix}{i}'
                if prob_col in df_arch.columns:
                    df_arch[prob_col] = df_arch[prob_col].apply(ast.literal_eval)
            
            all_models_data.append(df_arch)
    
    if not all_models_data:
        raise ValueError("Nenhum dado de arquitetura foi encontrado")
    
    df_ensemble = all_models_data[0][['patient_id', 'tile_name', 'true_label']].copy()
    
    all_probs = []
    for df_arch in all_models_data:
        for i in range(10):
            for prefix in ['g', 'm', 's']:
                prob_col = f'probability_vector_{prefix}{i}'
                if prob_col in df_arch.columns:
                    probs_series = df_arch[prob_col].apply(
                        lambda x: x if isinstance(x, dict) else {}
                    )
                    all_probs.append(probs_series)
    
    mean_probs_list = []
    true_probs_list = []
    predicted_labels_list = []
    predicted_probs_list = []
    
    for idx in range(len(df_ensemble)):
        tile_probs = []
        for probs_series in all_probs:
            if idx < len(probs_series) and probs_series.iloc[idx]:
                tile_probs.append(probs_series.iloc[idx])
        
        if tile_probs:
            mean_probs = {
                'cin': np.mean([p.get('cin', 0) for p in tile_probs]),
                'ebv': np.mean([p.get('ebv', 0) for p in tile_probs]),
                'msi': np.mean([p.get('msi', 0) for p in tile_probs]),
                'gs': np.mean([p.get('gs', 0) for p in tile_probs])
            }
            
            predicted_label = max(mean_probs, key=mean_probs.get)
            predicted_prob = mean_probs[predicted_label]
            
            true_label = df_ensemble.iloc[idx]['true_label']
            true_prob = mean_probs.get(true_label, 0)
            
            mean_probs_list.append(mean_probs)
            true_probs_list.append(true_prob)
            predicted_labels_list.append(predicted_label)
            predicted_probs_list.append(predicted_prob)
        else:
            mean_probs_list.append({})
            true_probs_list.append(0)
            predicted_labels_list.append('')
            predicted_probs_list.append(0)
    
    df_ensemble['mean_probs_per_class_tile'] = mean_probs_list
    df_ensemble['true_prob'] = true_probs_list
    df_ensemble['predicted_label'] = predicted_labels_list
    df_ensemble['predicted_probability'] = predicted_probs_list
    
    return df_ensemble

# Função para calcular métricas por classe usando bootstrapping
def calculate_metrics_with_bootstrapping(df, n_bootstraps=100):
    """Calcula métricas usando bootstrapping para estimar variabilidade"""
    bootstrap_metrics = {
        'accuracy': [],
        'precision_cin': [], 'precision_ebv': [], 'precision_msi': [], 'precision_gs': [], 'precision_weighted': [],
        'recall_cin': [], 'recall_ebv': [], 'recall_msi': [], 'recall_gs': [], 'recall_weighted': [],
        'f1_cin': [], 'f1_ebv': [], 'f1_msi': [], 'f1_gs': [], 'f1_weighted': []
    }
    
    for _ in range(n_bootstraps):
        bootstrap_sample = resample(df, replace=True, random_state=_)
        
        accuracy = accuracy_score(bootstrap_sample['true_label'], bootstrap_sample['predicted_label'])
        precision = precision_score(bootstrap_sample['true_label'], bootstrap_sample['predicted_label'], 
                                  average=None, labels=['cin', 'ebv', 'msi', 'gs'], zero_division=0)
        recall = recall_score(bootstrap_sample['true_label'], bootstrap_sample['predicted_label'], 
                             average=None, labels=['cin', 'ebv', 'msi', 'gs'], zero_division=0)
        f1 = f1_score(bootstrap_sample['true_label'], bootstrap_sample['predicted_label'], 
                     average=None, labels=['cin', 'ebv', 'msi', 'gs'], zero_division=0)
        
        precision_weighted = precision_score(bootstrap_sample['true_label'], bootstrap_sample['predicted_label'], 
                                           average='weighted', zero_division=0)
        recall_weighted = recall_score(bootstrap_sample['true_label'], bootstrap_sample['predicted_label'], 
                                     average='weighted', zero_division=0)
        f1_weighted = f1_score(bootstrap_sample['true_label'], bootstrap_sample['predicted_label'], 
                             average='weighted', zero_division=0)
        
        bootstrap_metrics['accuracy'].append(accuracy)
        
        bootstrap_metrics['precision_cin'].append(precision[0])
        bootstrap_metrics['precision_ebv'].append(precision[1])
        bootstrap_metrics['precision_msi'].append(precision[2])
        bootstrap_metrics['precision_gs'].append(precision[3])
        bootstrap_metrics['precision_weighted'].append(precision_weighted)
        
        bootstrap_metrics['recall_cin'].append(recall[0])
        bootstrap_metrics['recall_ebv'].append(recall[1])
        bootstrap_metrics['recall_msi'].append(recall[2])
        bootstrap_metrics['recall_gs'].append(recall[3])
        bootstrap_metrics['recall_weighted'].append(recall_weighted)
        
        bootstrap_metrics['f1_cin'].append(f1[0])
        bootstrap_metrics['f1_ebv'].append(f1[1])
        bootstrap_metrics['f1_msi'].append(f1[2])
        bootstrap_metrics['f1_gs'].append(f1[3])
        bootstrap_metrics['f1_weighted'].append(f1_weighted)
    
    result = {}
    for metric, values in bootstrap_metrics.items():
        if values:
            result[f'{metric}_mean'] = np.mean(values)
            result[f'{metric}_std'] = np.std(values, ddof=1)
    
    return result

# Função para preparar dados para teste de Wilcoxon
def prepare_wilcoxon_data(df_sa, df_ma):
    """Prepara dados para teste de Wilcoxon por classe"""
    # Adicionar identificador único para cada tile
    df_sa['tile_id'] = df_sa['patient_id'] + '_' + df_sa['tile_name']
    df_ma['tile_id'] = df_ma['patient_id'] + '_' + df_ma['tile_name']
    
    # Unir os dataframes por tile_id
    df_common = pd.merge(
        df_sa[['tile_id', 'true_label', 'predicted_label', 'true_prob']].rename(columns={
            'true_prob': 'true_prob_sa', 
            'predicted_label': 'predicted_label_sa'
        }),
        df_ma[['tile_id', 'true_label', 'predicted_label', 'true_prob']].rename(columns={
            'true_prob': 'true_prob_ma', 
            'predicted_label': 'predicted_label_ma'
        }),
        on=['tile_id', 'true_label'], 
        how='inner'
    )
    
    return df_common

# Função para executar teste de Wilcoxon por classe
def run_wilcoxon_tests(df_common):
    """Executa testes de Wilcoxon para cada classe e métrica"""
    results = {}
    classes = ['cin', 'ebv', 'msi', 'gs']
    
    for cls in classes:
        # Filtrar apenas tiles da classe atual
        df_cls = df_common[df_common['true_label'] == cls]
        
        if len(df_cls) > 0:
            # Calcular acurácia por tile (1 se correto, 0 se incorreto)
            df_cls['accuracy_sa'] = (df_cls['predicted_label_sa'] == cls).astype(int)
            df_cls['accuracy_ma'] = (df_cls['predicted_label_ma'] == cls).astype(int)
            
            # Executar teste de Wilcoxon para acurácia
            stat_accuracy, p_value_accuracy = wilcoxon(
                df_cls['accuracy_sa'], 
                df_cls['accuracy_ma']
            )
            
            # Executar teste de Wilcoxon para true_prob
            stat_true_prob, p_value_true_prob = wilcoxon(
                df_cls['true_prob_sa'], 
                df_cls['true_prob_ma']
            )
            
            results[cls] = {
                'accuracy': {'stat': stat_accuracy, 'p_value': p_value_accuracy},
                'true_prob': {'stat': stat_true_prob, 'p_value': p_value_true_prob}
            }
    
    return results

# Processar dados
print("Processando dados do SA (EffNet)...")
df_sa = process_sa_data(SA_FOLDS_DIR)

print("Construindo tabela ensemble do MA...")
df_ma = build_ensemble_table(MA_MODELS_DIR)

# Calcular métricas com bootstrapping
print("Calculando métricas com bootstrapping...")
sa_bootstrap_metrics = calculate_metrics_with_bootstrapping(df_sa, N_BOOTSTRAPS)
ma_bootstrap_metrics = calculate_metrics_with_bootstrapping(df_ma, N_BOOTSTRAPS)

# Preparar dados para teste de Wilcoxon
print("Preparando dados para teste de Wilcoxon...")
df_common = prepare_wilcoxon_data(df_sa, df_ma)

# Executar testes de Wilcoxon
print("Executando testes de Wilcoxon...")
wilcoxon_results = run_wilcoxon_tests(df_common)

# Função auxiliar para formatar valores
def format_value(value, format_str="{:.4f}"):
    if isinstance(value, (int, float, np.number)):
        return format_str.format(value)
    else:
        return str(value)

# Função específica para formatar valor-p com mais casas decimais
def format_p_value(value):
    if isinstance(value, (int, float, np.number)):
        return "{:.6f}".format(value)
    else:
        return str(value)

# Gerar tabelas de resultados
classes = ['cin', 'ebv', 'msi', 'gs', 'weighted']
metrics = ['precision', 'recall', 'f1']

print("\n" + "="*80)
print("TABELA DE RESULTADOS - PRECISION")
print("="*80)
print("Classe\t\tEFFNet (SA)\t\tENSEMBLE (MA)\t\tEstatística W\tValor-p\tSignificância")
for cls in classes:
    sa_mean = sa_bootstrap_metrics.get(f'precision_{cls}_mean', np.nan)
    sa_std = sa_bootstrap_metrics.get(f'precision_{cls}_std', np.nan)
    ma_mean = ma_bootstrap_metrics.get(f'precision_{cls}_mean', np.nan)
    ma_std = ma_bootstrap_metrics.get(f'precision_{cls}_std', np.nan)
    
    if cls in wilcoxon_results and cls != 'weighted':
        stat = wilcoxon_results[cls]['accuracy']['stat']
        p_value = wilcoxon_results[cls]['accuracy']['p_value']
        sig = "Sim" if p_value < 0.05 else "Não"
    else:
        stat, p_value, sig = "N/A", "N/A", "N/A"
    
    if not np.isnan(sa_mean) and not np.isnan(ma_mean):
        print(f"{cls}\t\t{format_value(sa_mean)} ± {format_value(sa_std)}\t{format_value(ma_mean)} ± {format_value(ma_std)}\t{format_value(stat)}\t{format_p_value(p_value)}\t{sig}")

print("\n" + "="*80)
print("TABELA DE RESULTADOS - RECALL")
print("="*80)
print("Classe\t\tEFFNet (SA)\t\tENSEMBLE (MA)\t\tEstatística W\tValor-p\tSignificância")
for cls in classes:
    sa_mean = sa_bootstrap_metrics.get(f'recall_{cls}_mean', np.nan)
    sa_std = sa_bootstrap_metrics.get(f'recall_{cls}_std', np.nan)
    ma_mean = ma_bootstrap_metrics.get(f'recall_{cls}_mean', np.nan)
    ma_std = ma_bootstrap_metrics.get(f'recall_{cls}_std', np.nan)
    
    if cls in wilcoxon_results and cls != 'weighted':
        stat = wilcoxon_results[cls]['accuracy']['stat']
        p_value = wilcoxon_results[cls]['accuracy']['p_value']
        sig = "Sim" if p_value < 0.05 else "Não"
    else:
        stat, p_value, sig = "N/A", "N/A", "N/A"
    
    if not np.isnan(sa_mean) and not np.isnan(ma_mean):
        print(f"{cls}\t\t{format_value(sa_mean)} ± {format_value(sa_std)}\t{format_value(ma_mean)} ± {format_value(ma_std)}\t{format_value(stat)}\t{format_p_value(p_value)}\t{sig}")

print("\n" + "="*80)
print("TABELA DE RESULTADOS - F1-SCORE")
print("="*80)
print("Classe\t\tEFFNet (SA)\t\tENSEMBLE (MA)\t\tEstatística W\tValor-p\tSignificância")
for cls in classes:
    sa_mean = sa_bootstrap_metrics.get(f'f1_{cls}_mean', np.nan)
    sa_std = sa_bootstrap_metrics.get(f'f1_{cls}_std', np.nan)
    ma_mean = ma_bootstrap_metrics.get(f'f1_{cls}_mean', np.nan)
    ma_std = ma_bootstrap_metrics.get(f'f1_{cls}_std', np.nan)
    
    if cls in wilcoxon_results and cls != 'weighted':
        stat = wilcoxon_results[cls]['accuracy']['stat']
        p_value = wilcoxon_results[cls]['accuracy']['p_value']
        sig = "Sim" if p_value < 0.05 else "Não"
    else:
        stat, p_value, sig = "N/A", "N/A", "N/A"
    
    if not np.isnan(sa_mean) and not np.isnan(ma_mean):
        print(f"{cls}\t\t{format_value(sa_mean)} ± {format_value(sa_std)}\t{format_value(ma_mean)} ± {format_value(ma_std)}\t{format_value(stat)}\t{format_p_value(p_value)}\t{sig}")

print("\n" + "="*80)
print("TABELA DE RESULTADOS - ACURÁCIA E TRUE_PROB")
print("="*80)
print("Métrica\t\tClasse\t\tEFFNet (SA)\t\tENSEMBLE (MA)\t\tEstatística W\tValor-p\tSignificância")

# Acurácia geral
sa_acc_mean = sa_bootstrap_metrics.get('accuracy_mean', np.nan)
sa_acc_std = sa_bootstrap_metrics.get('accuracy_std', np.nan)
ma_acc_mean = ma_bootstrap_metrics.get('accuracy_mean', np.nan)
ma_acc_std = ma_bootstrap_metrics.get('accuracy_std', np.nan)

# Calcular estatísticas de Wilcoxon para acurácia geral
df_common['accuracy_sa'] = (df_common['predicted_label_sa'] == df_common['true_label']).astype(int)
df_common['accuracy_ma'] = (df_common['predicted_label_ma'] == df_common['true_label']).astype(int)

stat_acc, p_value_acc = wilcoxon(df_common['accuracy_sa'], df_common['accuracy_ma'])
sig_acc = "Sim" if p_value_acc < 0.05 else "Não"

print(f"Acurácia\tOverall\t\t{format_value(sa_acc_mean)} ± {format_value(sa_acc_std)}\t{format_value(ma_acc_mean)} ± {format_value(ma_acc_std)}\t{format_value(stat_acc)}\t{format_p_value(p_value_acc)}\t{sig_acc}")

# True_prob geral
stat_true_prob, p_value_true_prob = wilcoxon(df_common['true_prob_sa'], df_common['true_prob_ma'])
sig_true_prob = "Sim" if p_value_true_prob < 0.05 else "Não"

sa_true_prob_mean = np.mean(df_common['true_prob_sa'])
sa_true_prob_std = np.std(df_common['true_prob_sa'], ddof=1)
ma_true_prob_mean = np.mean(df_common['true_prob_ma'])
ma_true_prob_std = np.std(df_common['true_prob_ma'], ddof=1)

print(f"True_prob\tOverall\t\t{format_value(sa_true_prob_mean)} ± {format_value(sa_true_prob_std)}\t{format_value(ma_true_prob_mean)} ± {format_value(ma_true_prob_std)}\t{format_value(stat_true_prob)}\t{format_p_value(p_value_true_prob)}\t{sig_true_prob}")

# True_prob por classe
for cls in ['cin', 'ebv', 'msi', 'gs']:
    df_cls = df_common[df_common['true_label'] == cls]
    if len(df_cls) > 0:
        stat, p_value = wilcoxon(df_cls['true_prob_sa'], df_cls['true_prob_ma'])
        sig = "Sim" if p_value < 0.05 else "Não"
        
        sa_mean = np.mean(df_cls['true_prob_sa'])
        sa_std = np.std(df_cls['true_prob_sa'], ddof=1)
        ma_mean = np.mean(df_cls['true_prob_ma'])
        ma_std = np.std(df_cls['true_prob_ma'], ddof=1)
        
        print(f"True_prob\t{cls}\t\t{format_value(sa_mean)} ± {format_value(sa_std)}\t{format_value(ma_mean)} ± {format_value(ma_std)}\t{format_value(stat)}\t{format_p_value(p_value)}\t{sig}")

# Salvar a tabela ensemble para uso futuro
ensemble_output_path = os.path.join(MA_MODELS_DIR, 'ensemble_per_tile_soft_voting.csv')
df_ma.to_csv(ensemble_output_path, index=False)
print(f"\nTabela ensemble salva em: {ensemble_output_path}")

Processando dados do SA (EffNet)...
Construindo tabela ensemble do MA...
Calculando métricas com bootstrapping...
Preparando dados para teste de Wilcoxon...
Executando testes de Wilcoxon...

TABELA DE RESULTADOS - PRECISION
Classe		EFFNet (SA)		ENSEMBLE (MA)		Estatística W	Valor-p	Significância
cin		0.5388 ± 0.0006	0.6248 ± 0.0014	709917264.0000	0.000000	Sim
ebv		0.1030 ± 0.0007	0.6242 ± 0.0037	14258959.5000	0.000000	Sim
msi		0.3844 ± 0.0008	0.6450 ± 0.0038	278709444.0000	0.000000	Sim
gs		0.1650 ± 0.0008	0.3362 ± 0.0027	7806509.5000	0.000000	Sim
weighted		0.3840 ± 0.0005	0.5853 ± 0.0013	N/A	N/A	N/A

TABELA DE RESULTADOS - RECALL
Classe		EFFNet (SA)		ENSEMBLE (MA)		Estatística W	Valor-p	Significância
cin		0.5968 ± 0.0005	0.8048 ± 0.0014	709917264.0000	0.000000	Sim
ebv		0.0957 ± 0.0007	0.3906 ± 0.0030	14258959.5000	0.000000	Sim
msi		0.3739 ± 0.0007	0.2819 ± 0.0023	278709444.0000	0.000000	Sim
gs		0.1305 ± 0.0006	0.3970 ± 0.0031	7806509.5000	0.000000	Sim
weighted		0.4014 ± 0.0004	0.5751 ± 

C:\Users\Fernando Alves\AppData\Local\Temp\ipykernel_17416\2920389596.py:218: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cls['accuracy_sa'] = (df_cls['predicted_label_sa'] == cls).astype(int)
C:\Users\Fernando Alves\AppData\Local\Temp\ipykernel_17416\2920389596.py:219: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cls['accuracy_ma'] = (df_cls['predicted_label_ma'] == cls).astype(int)
C:\Users\Fernando Alves\AppData\Local\Temp\ipykernel_17416\2920389596.py:218: SettingWithCopyWarning: 
A value is tr

Acurácia	Overall		0.4014 ± 0.0004	0.5751 ± 0.0011	3186877723.0000	0.000000	Sim
True_prob	Overall		0.4784 ± 0.3421	0.4670 ± 0.2262	41720264207.5000	0.000000	Sim
True_prob	cin		0.6258 ± 0.3002	0.5894 ± 0.1781	12668416663.5000	0.000000	Sim
True_prob	ebv		0.1228 ± 0.1905	0.2930 ± 0.1951	104147723.5000	0.000000	Sim
True_prob	msi		0.3573 ± 0.2933	0.2996 ± 0.1495	2473763013.0000	0.000000	Sim
True_prob	gs		0.1071 ± 0.1764	0.2680 ± 0.1781	64673041.0000	0.000000	Sim

Tabela ensemble salva em: C:\Users\Fernando Alves\Documents\WANG PIPELINE\Wang\MA_models\ensemble_per_tile_soft_voting.csv
